# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering.

In [1]:
import pandas as pd
from sklearn import linear_model

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
kc_house_train = pd.read_csv('~/Courses/u-wash-machine-learning/regression/data/kc_house_train_data.csv')
kc_house_test  = pd.read_csv('~/Courses/u-wash-machine-learning/regression/data/kc_house_test_data.csv')

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [3]:
# Create linear regression object
example_model = linear_model.LinearRegression()

In [63]:
# Set Features/Target
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_target = ['price']

# Set up Data
example_X_train = kc_house_train[example_features]
example_y_train = kc_house_train[example_target]

example_X_test  = kc_house_test[example_features]
example_y_test  = kc_house_test[example_target]

# Fit Linear Regression
example_model.fit(example_X_train, example_y_train)

LinearRegression()

Now that we have fitted the model we can extract the regression weights (coefficients) as follows:

In [43]:
print('Coefficients: \n', example_model.coef_)

Coefficients: 
 [[   315.40669062 -65081.88711588   6942.16598637]]


In [44]:
print('Intercept: \n', example_model.intercept_)

Intercept: 
 [87912.86581496]


# Making Predictions

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [60]:
y_pred = example_model.predict(example_X_train)
y_pred[0] # should be 271789

array([271789.26537997])

# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [68]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    y_pred = model.predict(data)

    # Then compute the residuals/errors
    residuals = outcome - y_pred

    # Then square and add them up
    RSS = (residuals * residuals).sum()

    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [67]:
rss_example_train = get_residual_sum_of_squares(example_model, example_X_test, example_y_test)
print(rss_example_train) # should be 2.7376153833e+14

price    2.737619e+14
dtype: float64


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [90]:
from math import log
import numpy as np

In [110]:
# Set Features/Target
example_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
example_target = ['price']

# Set up Data
example_X_train = kc_house_train[example_features]
example_y_train = kc_house_train[example_target]

example_X_test  = kc_house_test[example_features]
example_y_test  = kc_house_test[example_target]

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [111]:
#example_X_train['bedrooms_squared'] = example_X_train['bedrooms'].apply(lambda x: x**2)
example_X_train = example_X_train.assign(bedrooms_squared = lambda x: (x['bedrooms']**2))
example_X_test = example_X_test.assign(bedrooms_squared = lambda x: (x['bedrooms']**2))

In [112]:
# create the remaining 3 features in both TEST and TRAIN data
example_X_train = example_X_train.assign(bed_bath_rooms = lambda x: (x['bedrooms'] * x['bathrooms']))
example_X_test = example_X_test.assign(bed_bath_rooms = lambda x: (x['bedrooms'] * x['bathrooms']))

example_X_train = example_X_train.assign(log_sqft_living = lambda x: (np.log(x['sqft_living'])))
example_X_test = example_X_test.assign(log_sqft_living = lambda x: (np.log(x['sqft_living'])))

example_X_train = example_X_train.assign(lat_plus_long = lambda x: (x['lat'] + x['long']))
example_X_test = example_X_test.assign(lat_plus_long = lambda x: (x['lat'] + x['long']))

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [113]:
example_X_test.mean()

sqft_living         2079.366280
bedrooms               3.376921
bathrooms              2.113561
lat                   47.563092
long                -122.216426
bedrooms_squared      12.446678
bed_bath_rooms         7.503902
log_sqft_living        7.550275
lat_plus_long        -74.653334
dtype: float64

# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [114]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using turicreate.linear_regression.create() and look at the value of the weights/coefficients:

In [115]:
# Learn the three models: (don't forget to set validation_set = None)
model1 = linear_model.LinearRegression()
model2 = linear_model.LinearRegression()
model3 = linear_model.LinearRegression()

# Fit Linear Regressions
model1.fit(example_X_train[model_1_features], example_y_train)
model2.fit(example_X_train[model_2_features], example_y_train)
model3.fit(example_X_train[model_3_features], example_y_train)

LinearRegression()

In [116]:
# Examine/extract each model's coefficients:
print('Coefficients: \n', model1.coef_)
print('Coefficients: \n', model2.coef_)
print('Coefficients: \n', model3.coef_)

Coefficients: 
 [[ 3.12258646e+02 -5.95865332e+04  1.57067421e+04  6.58619264e+05
  -3.09374351e+05]]
Coefficients: 
 [[ 3.06610053e+02 -1.13446368e+05 -7.14613083e+04  6.54844630e+05
  -2.94298969e+05  2.55796520e+04]]
Coefficients: 
 [[ 5.29422820e+02  3.45142296e+04  6.70607813e+04  5.34085611e+05
  -4.06750711e+05 -8.57050439e+03 -6.78858667e+03 -5.61831484e+05
   1.27334900e+05]]


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**
<br>1.57067421e+04  
<br>
**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**
<br>-7.14613083e+04
<br>Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [121]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
get_residual_sum_of_squares(model1, example_X_train[model_1_features], example_y_train)

price    9.678800e+14
dtype: float64

In [122]:
get_residual_sum_of_squares(model2, example_X_train[model_2_features], example_y_train)

price    9.584196e+14
dtype: float64

In [123]:
get_residual_sum_of_squares(model3, example_X_train[model_3_features], example_y_train)

price    9.034365e+14
dtype: float64

**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?
<br>
Model 3

Now compute the RSS on on TEST data for each of the three models.

In [124]:
# Compute the RSS on TESTING data for each of the three models and record the values:
get_residual_sum_of_squares(model1, example_X_test[model_1_features], example_y_test)

price    2.255005e+14
dtype: float64

In [126]:
get_residual_sum_of_squares(model2, example_X_test[model_2_features], example_y_test)

price    2.233775e+14
dtype: float64

In [125]:
get_residual_sum_of_squares(model3, example_X_test[model_3_features], example_y_test)

price    2.592363e+14
dtype: float64

**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.
<br>
Model 2